## Redes recurrentes


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 0: Definición de la red  </strong></div>

Contesta a las siguientes preguntas sobre la clase `RNN(nn.Module)` que hemos definido en el boletín

* La variable `input_size` en el constructor ¿a qué se refiere exactamente?

* Sea el siguiente código que usamos para definir una red recurrente 

```python
class RNN(nn.Module):
    # Como podemos ver, ahora gestionamos el estado oculto
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        #La red es un MLP de dos capas
        #Hasta aquí, idéntico a un MLP
        self.i2ha = nn.Linear(input_size, hidden_size)
        self.ha2hb = nn.Linear(hidden_size, hidden_size)
        self.hb2hc = nn.Linear(hidden_size, hidden_size)
        self.hc2hd = nn.Linear(hidden_size, hidden_size)
        self.hd2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #En la variable hidden es donde se genera el estado oculto
        #que se usa como recurrencia
        hidden = F.tanh(self.i2ha(input) + self.ha2hb(hidden))
        output = self.hd2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
```
Enumera las diferencias de esta red en cuanto a funcionamiento y nivel de resultados con respecto a la que hemos visto en el boletín Lab4.

* Dado el código anterior, reprograma el método `forward()` para que se consideren las 5 capas lineales y siga siendo una red recurrente. 

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 0: Respuestas  </strong></div>

* Pregunta 1: la variable `input_size` en el constructor se refiere al número de componentes del vector de entrada a la red. La red neuronal que hemos definido procesaba nombres. Para la RNN, una sentencia es el nombre. Y, por tanto, los tokens de la sentencia son los caracteres que componen el nombre. Y cada uno de esos tokens, como sabemos, se procesa con una llamada `forward()` por token. Por tanto, `input_size` se refiere al número de bits que usamos para representar, usando one hot, un carácter ASCII.

* Pregunta 2: la diferencia principal entre la red que aquí aparece y la que hemos visto en clase tiene que ver con la estructura. En la red vista en clase, teníamos un MLP de dos capas (i.e. capa de nodos entre los de salida y los de entrada y otra de nodos a la salida hacen dos capas, nunca se cuenta la capa de entrada). En este caso tenemos un MLP de cuatro capas. En la red vista en clase, la recurrencia ocurría en la capa oculta. La salida de la capa oculta se alimentaba con la salida de la misma llamada en el instante anterior. ¿Cómo sabemos eso? Porque mirábamos a cómo estaba programado la función `forward()`. Si nos fijamos en esa versión, vemos que el valor que toma `hidden` en el código interno de `forward()` depende de cómo pasa `input` por la capa de entrada y `hidden` (en el instante anterior) por la capa oculta.

```python
def forward(self, input, hidden):
        #En la variable hidden es donde se genera el estado oculto
        #que se usa como recurrencia
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden        
```

En la versión actual, es idéntica. De hecho, las variables `hb2hc` y `hc2hd` están puestas para despistar. ¿Por qué? Pues porque no se usan. Si no se hace uso de ellas en `forward()` no se usan en el grafo de computación. Y, por lo tanto, aunque se hayan definido como variables de clase, no juegan papel alguno.

* Pregunta 3: 

```python
def forward(self, input, hidden):
        #En la variable hidden es donde se genera el estado oculto
        #que se usa como recurrencia
        hidden = F.tanh(self.i2ha(input) + self.ha2hb(hidden))
        hiddenb = self.hb2hc(hidden)
        hiddenb = self.hc2hd(hiddenb)
        output = self.hd2o(hiddenb)
        output = self.softmax(output)
        return output, hidden       
```
En esta implementación, hacemos que la primera matriz de pesos, que contiene los correspondientes a los arcos desde la entrada a la primera capa oculta, sea la única que depende de los valores de $h_t$ y, por tanto, sea la que module su tratamiento.


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 1: ¿Por qué aprende la red?</strong></div>

* Trata de razonar sobre cuál es la verdadera naturaleza del mecanismo mediante el cual la red aprende. Intenta elaborar una explicación que teorice acerca de qué descubre sobre los nombres para determinar a qué lengua pertenecen. 

* ¿Qué heurística usarías tú para decidir si un nombre que no has visto nunca se relaciona con una lengua concreta? ¿Tiene algo que ver? 

* ¿Cómo verificarías tu teoría sobre cómo funciona la RNN en este problema?

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 1: Respuestas</strong></div>

* Pregunta 1: la única información de la que dispone la red es, para cada nombre, de la secuencia de caracteres que forman el nombre, de izquierda a derecha. Nada más. No hay nada implicado de naturaleza fonémica porque no hay nada que tenga que ver con los sonidos de los propios lenguajes considerados aquí. Son simplemente caracteres ASCII. Por tanto, la RNN se esfuera en buscar subgrupos de caracteres que suelen ir juntos, seguramente formando patrones al estilo de expresiones regulares.

* Pregunta 2: como humano, seguramente me apoyaría más en el sonido que en la secuencia de caracteres per sé. Pues, como sabemos, es más importante la sonoridad de la lengua que la secuencia de caracteres. Sin embargo, en este problema carecemos de esa información.

* Pregunta 3:  Si lo que quiero es verificar si la RNN es capaz de distinguir entre expresiones regulares, definiría un conjunto de expresiones regulares suficientemente complejas, de tal forma que cada una equivaldría a una lengua y las usaría para generar nombres pertenecientes a esa lengua. Si la RNN llega a ser capaz de distinguir ciertas variedades de expresiones regulares, sería capaz de modelarlas. 



<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 2: El tamaño de la capa recurrente</strong></div>

En el guión de prácticas, hemos usado una red neuronal de 128 nodos en la capa oculta. Prueba cuatro tamaños alternativos, de 64, 32, 256 y 512 nodos. Representa los resultados añadiendo a la curva de la pérdida las otras cuatro curvas. 

* ¿Qué se observa?

* A la luz de este grupo de curvas, ¿podrías escoger de manera más o menos informada una de entre las cinco como la mejor de todas? ¿por qué?

* Sugiere, con cambios sencillos, una nueva estrategia de evaluación para las cinco redes. Impleméntala y determina cuál es la mejor.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 2: Respuestas</strong></div>

* Pregunta 1: Este ejercicio no se va a resolver sino que se va a dejar indicado para que sea el alumno el que lo resuelva. Para ello, puede basarse en el ejercicio 4 del Laboratorio 2, ya resuelto. Las curvas se dispondrán a determinadas alturas, si representamos en el eje $x$ los lotes y en el eje $y$ la pérdida a lo largo del entrenamiento. Aquellas curvas con mayor pérdida necesitarán más lotes para aprender y se corresponderán, normalmente, con tamaños más grandes.

* Pregunta 2: ciertamente cualquier red con una función de pérdida que muestre una tendencia decreciente podría ser válida a priori. Sin embargo, esto no es suficiente. 

* Pregunta 3: Es necesario primero reservar ejemplos fuera del entrenamiento, para una validación basada en ejemplos no vistos en tiempo de entrenamiento. Y por el otro, también es necesario definir un estadístico (i.e., una medida) propio del problema que estamos tratanto. En este caso, una tasa de aciertos (i.e., número de aciertos/número de preguntas). Aquel modelo que muestre una mejor tasa de aciertos en ejemplos no vistos en el entrenamiento podría ser un candidato. 





<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 3: La definición de red recurrente</strong></div>

Echa un vistazo a la implementación de la clase correspondiente a la red recurrente tipo Elman disponible en Pytorch, cuya documentación se encuentra [aquí](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)

* Fíjate en la implementación del método `forward()`: indica, si es que hay alguna, las diferencias en la inferencia de esta red y la que hemos definido nosotros.

* Estudia la documentación de las otras dos tipos de redes de las que se dispone en Pytorch, [LSTMs](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) y [GRUs](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html). 

    - Verifica que los tres tipos de red tienen exactamente las mismas entradas y las mismas salidas

    - Verifica que GRU son un tipo simplificado de las LSTM

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio 3: Respuestas</strong></div>

* Pregunta 1: el código de la función `forward()` de la documentación que allí aparece es este (he eliminado alguna parte totalmente superflua)

```python
def forward(x, h_0=None):
    seq_len, batch_size, _ = x.size()
    if h_0 is None:
        h_0 = torch.zeros(num_layers, batch_size, hidden_size)
    h_t_minus_1 = h_0
    h_t = h_0
    output = []
    for t in range(seq_len):
        for layer in range(num_layers):
            h_t[layer] = torch.tanh(
                x[t] @ weight_ih[layer].T
                + bias_ih[layer]
                + h_t_minus_1[layer] @ weight_hh[layer].T
                + bias_hh[layer]
            )
        output.append(h_t[-1])
        h_t_minus_1 = h_t
    output = torch.stack(output)
    return output, h_t
```

Como vemos, es algo más flexible que el que hemos programado nosotros pero en esencia similar. Primero nos quedamos con la longitud de la entrada, y el número de secuencias (i.e., el tamaño de batch). Si no recibe como segundo argumento el valor del estado oculto, significa que la hemos llamado por primera vez y, entonces, inicializamos $h_0$ con un tensor de ceros. Luego inicializamos una lista `output`. Cada elemento de esa lista será un $h_t$ ya que procesaremos cada token de la secuencia internamente en este caso. Es decir, no haremos una llamada por cada token sino que haremos una sola llamada por lote, siendo cada lote un conjunto de secuencias de longitud fija y cada secuencia un número de tokens. Además, cada token seguirá un proceso secuencial através de un número `num_layer` de capas variable.  En todo caso, la recurrencia sigue ocurriendo gracias a la expresión 

```python
h_t[layer] = torch.tanh(x[t] @ weight_ih[layer].T + bias_ih[layer] + h_t_minus_1[layer] @ weight_hh[layer].T + bias_hh[layer])
```
que está diciendo básicamente que $$tahn(x_t * U^l_{ih} + b^l_{ih} + h^l_{t-1} * W^l_{hh} + b_{hh}).$$ en donde el superíndice $l$ hace referencia a la *layer* de la red recurrente en la que se está trabajando. Por otro lado, si recordamos cómo se había definido nuestra red recurrente básica, en nuestra RNN, simplemente teníamos $$tahn(x_t * U_{ih} + b_{ih} + h_{t-1} * W_{hh} + b_{hh}),$$ porque no teníamos capas apiladas. No hay prácticamente diferencia.

* Pregunta 2: si nos fijamos en la documentación de cada una de las tres tredes, la primera ecuación para cada una de las tres RNNs es, respectivamente

    - Elman RNN: $tanh(x_tW^T_{ih} + b_{ih} + h_{t-1}W_{hh}^T + b_{hh})$

    - LSTM RNN: 
        
        + $i_t=\sigma(W{ii}x_t + b_{ii} + W_{hi}h_{t-1} + b_{hi}),$ 

        + $f_t=\sigma(W{if}x_t + b_{if} + W_{hf}h_{t-1} + b_{hf}),$

        + $g_t=\tanh(W{ig}x_t + b_{ig} + W_{hg}h_{t-1} + b_{hg}),$

        + $o_t=\sigma(W{io}x_t + b_{io} + W_{ho}h_{t-1} + b_{ho}),$

        + $c_t=f_t \cdot c_{t-1} + i_t \cdot g_t$, $h_t=o_t\cdot \tanh(c_t).$
    
    - GRU RNN: 
        
        + $r_t=\sigma(W{ir}x_t + b_{ir} + W_{hr}h_{t-1} + b_{hr}),$ 

        + $z_t=\sigma(W{iz}x_t + b_{iz} + W_{hz}h_{t-1} + b_{hz}),$

        + $n_t=\tanh(W{in}x_t + b_{in} + r_t\cdot (W_{hn}h_{t-1} + b_{hg}),$

        + $h_t=(1-z_t)\cdot n_t + z_t \cdot h_{t-1}.$

Como podemos ver, las tres redes tienen las mismas entradadas. Esas entradas son siempre $x_t$ y $h_{t-1}$. En la Elman ya lo sabíamos. En la LSTM, las ecuaciones para $i_t$, $f_t$, $g_t$ y $o_t$ pueden ocurrir en paralelo y las cuatro necesitan las mismas entradas, ese par que hemos mencionado. El resultado de esas cuatro ecuaciones se amalgama en $c_t$ finalmente. Y lo mismo ocurre con GRU y las ecuaciones $r_t$ y $z_t$. Tras esto, $n_t$ requiere de $r_t$ y $h_t$ reqwuiere de $n_t$ y $z_t$. Por tanto, las entradas al sistema son exactamente las mismas.  

* Pregunta 3: Las GRU se proponen por primera vez [aquí](https://arxiv.org/abs/1406.1078). En este paper, se menciona que se parte de las LSTM como idea pero que solamente se necesitan dos tipos de puertas, la puerta de reset $r_t$ y la de update $u_t$. Cuando la salida de la puerta de reset es cercana a 0, el estado oculto se fuerza a ignorar el valor del estado oculto anterior. La puerta de update indica cuánta información del estado anterior se va a usar en el estado actual (al igual que ocurría en la LSTM con la $c_t$). A simple vista puede verse que en términos de expresiones es más simple. Y necesita dos matrices de parámetros menos.

